# 01 Pytorch tensor
#### ＝＝＝ 目次 ＝＝＝
0. Pytorchの呼び出し
1. Tensorの生成
2. 基本演算
3. 誤差逆伝播&勾配降下法
4. GPUの使用

## Pytorchの特徴
- Numpyに代わってGPU上で動くパッケージ
- 柔軟性があり高速な深層学習のプラットフォーム
- define by run

pytorchはCUDAなどのバージョンを見ながら,インストールする必要がある

公式HP：https://pytorch.org/get-started/locally/

---
## 0. Pytorchの呼び出し
`torch`という名前のモジュールをインポートする

In [ ]:
import torch

---
## 1. Tensor(テンソル)の生成
tensor：多次元配列のようなもの(ex. ベクトル，行列)

Numpyのndarrayのようなもの(ndarrayと比べて，GPUを使うことで計算を高速化できる．また，勾配情報を保持できる．)

|<div align='center'>関数</div>|<div align='center'>意味</div>|<div align='center'>例</div>|
|---|---|---|
|<div align='left'>torch.tensor(array)</div>|<div align='left'>配列をtensorに変換</div>|<div align='left'>torch.tensor([2.5, 5.0, 3.6])</div>|
|<div align='left'>torch.empty(shape)</div>|<div align='left'>空のテンソルを作成 (何かしらの値が入っている)</div>|<div align='left'>torch.empty(2, 5)</div>|
|<div align='left'>torch.zeros(shape)</div>|<div align='left'>0のテンソルを作成</div>|<div align='left'>torch.zeros(2, 5)</div>|
|<div align='left'>torch.ones(shape)</div>|<div align='left'>1のテンソルを作成</div>|<div align='left'>torch.ones(2, 5)</div>|
|<div align='left'>torch.full(shape,fill_value)</div>|<div align='left'>任意の値のテンソルを作成</div>|<div align='left'>torch.full((2, 5),fill_value=4)</div>|
|<div align='left'>torch.zeros_like(tensor)</div>|<div align='left'>引数のテンソルと同じサイズの0のテンソルを作成</div>|<div align='left'>torch.zeros_like(a)</div>|
|<div align='left'>torch.eye(shape)</div>|<div align='left'>単位行列を作成</div>|<div align='left'>torch.eye(3, 3)</div>|
|<div align='left'>torch.rand(shape)</div>|<div align='left'>[0, 1]の一様分布による乱数</div>|<div align='left'>torch.rand(2, 5)</div>|
|<div align='left'>torch.randn(shape)</div>|<div align='left'>標準正規分布による乱数</div>|<div align='left'>torch.randn(2, 5)</div>|

`torch.tensor()`：配列をtensorに変換

`dtype`で値のデータ型を指定

In [ ]:
# 一次元配列
x = torch.tensor([5.5, 3, 2.4])
print(x)

In [ ]:
# 二次元配列
x = torch.tensor([[3, 5, 2],
                  [8, 8, 1],
                  [4, 1, 5],
                  [5, 8, 8],
                  [2, 5, 2]], dtype=torch.float)
print(x)

`torch.rand()`：[0, 1]の一様分布による乱数

In [ ]:
x = torch.rand(2, 5)
print(x)

`torch.randn()`：標準正規分布による乱数 (平均0, 分散1)

In [ ]:
x = torch.randn(2, 5)
print(x)

---
## tensorのshape
`変数.shape` or `変数.size()`：tensorのshapeを返す

In [ ]:
x = torch.randn(3, 6)
print(x.shape)
print(x.size())

`変数.view()`：tensorのshapeを変更したものを返す

In [ ]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8) # -1を使うと自動で調整してくれる
print("x :", x.shape)
print("y :", y.shape)
print("z :", z.shape)

`変数.squeeze(dim)`：指定した次元のサイズが1の場合削除、dimの指定がなければサイズ1をすべて削除

In [ ]:
x = torch.randn(1, 28, 28, 1)
y = x.squeeze(dim=0)
z = x.squeeze()
print("x :", x.shape)
print("y :", y.shape)
print("z :", z.shape)

`変数.unsqueeze(dim)`：指定した位置にサイズ1の次元を挿入したtensorを返す

In [ ]:
x = torch.randn(28, 28)
y = x.unsqueeze(dim=0)
z = x.unsqueeze(dim=1)
print("x :", x.shape)
print("y :", y.shape)
print("z :", z.shape)

---
## スライス
リストやndarrayのように，スライスを用いることで一部を抽出できる

In [ ]:
x = torch.randn(4, 6)
print(x)
print(x[1:3, :])

---
## Numpyとの変換
`変数.numpy()`：tensor → ndarray

In [ ]:
a = torch.randn(2, 3)
print(type(a))
print(a)

b = a.numpy()
print(type(b))
print(b)

`torch.from_numpy(ndarray)`：ndarray → tensor

In [ ]:
import numpy as np
a = np.ones(5)
print(type(a))
print(a)

b = torch.from_numpy(a)
print(type(b))
print(b)

---
## 2. 基本演算

|<div align='center'>演算</div>|<div align='center'>演算子</div>|
|---|---|
|<div align='center'>足し算</div>|<div align='center'>+</div>|
|<div align='center'>引き算</div>|<div align='center'>-</div>|
|<div align='center'>アダマール積</div>|<div align='center'>*</div>|
|<div align='center'>行列積$^{*1}$</div>|<div align='center'>torch.matmul()</div>|

$^{*1}$ Pytorchにはそれぞれのshapeのtensorに合わせた積の関数(`dot`や`mm`など)があるが，`matmul`は任意のshapeのtensorに対する汎用関数

In [ ]:
# テンソルの作成
x = torch.tensor([[4., 3.], 
                  [2., 1.]])
y = torch.tensor([[2., 2.], 
                  [1., 1.]])

In [ ]:
# 足し算
x + y

In [ ]:
# アダマール積
x * y

In [ ]:
# 行列積
torch.matmul(x, y)

In [ ]:
# 1次元 × 1次元 -> 0次元(スカラー)
x = torch.randn(3)
y = torch.randn(3)
z = torch.matmul(x, y)
print(z.shape)

In [ ]:
# 2次元 × 1次元 -> 1次元(ベクトル)
x = torch.randn(4, 3)
y = torch.randn(3)
z = torch.matmul(x, y)
print(z.shape)

In [ ]:
# 2次元 × 2次元 -> 2次元(行列)
x = torch.randn(4, 3)
y = torch.randn(3, 5)
z = torch.matmul(x, y)
print(z.shape)

In [ ]:
# 3次元 × 2次元 -> 3次元(テンソル)
x = torch.randn(100, 4, 3)
y = torch.randn(3, 5)
z = torch.matmul(x, y)
print(z.shape)

#### その他の演算

|<div align='center'>演算</div>|<div align='center'>関数</div>|
|---|---|
|<div align='center'>要素の和</div>|<div align='center'>torch.sum(tensor, dim)</div>|
|<div align='center'>要素の平均</div>|<div align='center'>torch.mean(tensor, dim)</div>|
|<div align='center'>要素の標準偏差</div>|<div align='center'>torch.std(tensor, dim)</div>|
|<div align='center'>要素の最大値</div>|<div align='center'>torch.max(tensor, dim)</div>|
|<div align='center'>要素の最小値</div>|<div align='center'>torch.min(tensor, dim)</div>|
|<div align='center'>tensorの結合</div>|<div align='center'>torch.cat(tensors, dim)</div>|

In [ ]:
# 要素の和
x = torch.ones(4, 3)
print(torch.sum(x))
print(torch.sum(x, dim=1))

In [ ]:
# 要素の最大値
x = torch.rand(2, 5)
torch.max(x)

max関数の`dim`を指定した場合，最大値を取るインデックスも返す

In [ ]:
x = torch.rand(100, 10)
max_values, indices = torch.max(x, dim=1)

print(max_values.shape)
print(max_values)
print(indices.shape)
print(indices)

`torch.cat(tensors, dim)`：指定した次元に対して，tensorを結合する

In [ ]:
tensor1 = torch.randn(100, 3, 10, 10)
tensor2 = torch.randn(100, 3, 10, 10)

torch.cat([tensor1, tensor2], dim=0).shape

In [ ]:
tensor1 = torch.randn(100, 3, 10, 10)
tensor2 = torch.randn(100, 3, 10, 10)

torch.cat([tensor1, tensor2], dim=1).shape

10個のtensor(3, 32, 32)を一つのtensor(10, 3, 32, 32)にまとめる

In [ ]:
# 10個のtensor(3, 32, 32)のリスト
tensors = [torch.randn(3, 32, 32) for _ in range(10)]

In [ ]:
tensors = torch.cat([tensor.unsqueeze(0) for tensor in tensors], dim=0)
tensors.shape

---
## 3. 誤差逆伝播&勾配降下法
`変数.backward()`：backpropagation(誤差逆伝播)による微分を行う．

`requires_grad=True`を指定することでtensorの勾配を保持する．

例.
$$z=x^2 + \frac{y^2}{2}$$
$$\frac{\partial z}{\partial x} = 2x, \frac{\partial z}{\partial y} = y$$
$$\frac{\partial z}{\partial x}|_{x=1.0} = 2.0, \frac{\partial z}{\partial y}|_{y=1.0} = 1.0$$

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(1.0, requires_grad=True)
z = x * x + y * y / 2 
print("x :", x)
print("y :", y)
print("z :", z)

In [ ]:
# backpropagation
z.backward()
print("xの勾配 :", x.grad)
print("yの勾配 :", y.grad)

## Optimizerによる勾配降下法
backpropagationで求めた勾配を用いて，optimizerによりtensorの値を更新する(学習)

例. $z=x^2 + \frac{y^2}{2}$が最小となるときの$(x,y)$を勾配降下法で求める．初期値 $(x,y)=(1.0,1.0)$

In [ ]:
# 初期値1のパラメータ
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(1.0, requires_grad=True)
print(x)
print(y)

In [ ]:
import torch.optim as optim

# optimizerを定義
# 更新するパラメータや学習率などを指定
optimizer = optim.SGD([x, y], lr=0.1)

SGDによる更新を1回行う
$$x=x-lr\frac{\partial z}{\partial x}, y=y-lr\frac{\partial z}{\partial y}$$

In [ ]:
optimizer.zero_grad() # 勾配を初期化
z = x*x + y*y/2       # 順伝播
z.backward()          # backpropagation
optimizer.step()      # 勾配を元にパラメータを更新

print(x)
print(y)

続けて更新を20回行う

In [ ]:
# 20回パラメータを更新
for i in range(1, 21):
    optimizer.zero_grad() # 勾配を初期化
    z = x*x + y*y/2       # 順伝播
    z.backward()          # backpropagation
    optimizer.step()      # 勾配を元にパラメータを更新

    print(i, ':', x, y)

$z=x^2 + \frac{y^2}{2}$が最小となるときの$(x,y)=(0,0)$に近づいてることが分かる

---
## 4. GPUの使用
`torch.cuda.is_available()`：GPU(cuda)が使用できる場合`True`を，できない場合`False`を返す

In [ ]:
torch.cuda.is_available()

使用しているデバイスをdeviceに代入する

cudaが使用可能であれば`"cuda:0"`，そうでなければ`"cpu"`を指定

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

`変数.to(device)`：変数をデバイス(cuda or cpu)に渡す

これによりデバイス上で計算が可能

In [ ]:
x = torch.randn(4)
y = torch.randn(4)

# tensorをGPUへ
x = x.to(device)
y = y.to(device)

z = x + y # GPU上で計算が行われる
print(z)

変数をCPUに渡す

In [ ]:
z = z.to("cpu")
print(z)

---
## マルチGPUの使用について
GPUが複数使用できる場合，`torch.nn.DataParallel`をモデルに適用することで並列計算が行える．

使用できるGPUの個数は`torch.cuda.device_count()`で確認できる．

In [ ]:
# modelを定義した後に記述
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)
    model.to(device)

ネットワークについては`02_pytorch_network.ipynb`で説明する．